# 🐦 BirdGuard — AI-Powered Tool for Legal & Ethical Hunting  
### Interactive Image Demo

This notebook is a **self-contained demo** of the BirdGuard system.

It applies **the same class + confidence logic** as the Pi scripts:
  - **Class 1** → illegal bird  
  - **Class 0** → legal bird  
  - Other classes are ignored for the LED decision  
- Displays an **LED indicator** that mimics the embedded LED:
  - 🔴 *blinking red* if any **illegal** bird is detected  
  - 🟢 *solid green* if no illegal bird but a **legal** bird is detected  
  - ⚪ *gray* if **no legal or illegal** bird is confidently detected  

**Upload an image and click “Run BirdGuard”.**

## 1. Environment Setup

This cell:

- Installs required Python packages (if missing).  
- Imports all necessary libraries.  

You only need to run it **once** at the start of the session.

In [ ]:
# 1. Environment Setup
# --------------------
# Ensure required packages are installed and import libraries.

import sys
import subprocess

def ensure_package(pkg_name: str):
    """Install a package via pip if it is not already available."""
    try:
        __import__(pkg_name)
        print(f"✅ '{pkg_name}' is already installed.")
    except ImportError:
        print(f"⏳ Installing '{pkg_name}' ...")
        subprocess.check_call([sys.executable, "-m", "pip", "install", pkg_name])
        print(f"✅ Installed '{pkg_name}'.")

# Core dependencies
ensure_package("ultralytics")
ensure_package("ipywidgets")
ensure_package("matplotlib")
ensure_package("opencv-python")  # matches repo usage

# Imports
from ultralytics import YOLO
from pathlib import Path
from PIL import Image
import io
import numpy as np

import ipywidgets as widgets
from IPython.display import display, HTML, clear_output

import matplotlib.pyplot as plt

print("✅ Environment is ready.")

⏳ Installing 'ultralytics' ...
✅ Installed 'ultralytics'.
✅ 'ipywidgets' is already installed.
✅ 'matplotlib' is already installed.
⏳ Installing 'opencv-python' ...
✅ Installed 'opencv-python'.
Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
✅ Environment is ready.


## 2. Clone the BirdGuard Repository & Locate the Model

The Raspberry Pi scripts load the YOLO model with:

```python
model = YOLO("pimodelv1.1.pt")
```

Here we:

- Clone `https://github.com/aazella/BirdGuard.git`  
- Confirm that `pimodelv1.1.pt` exists  
- Store its path in `MODEL_PATH`  

In [ ]:
# 2. Clone the BirdGuard GitHub repository
# ----------------------------------------

import os
import shutil
from pathlib import Path
import subprocess as _subprocess

repo_url = "https://github.com/aazella/BirdGuard.git"
repo_dir = "BirdGuard"

# If the repo already exists, remove it to avoid stale files
if os.path.exists(repo_dir):
    shutil.rmtree(repo_dir)

print("⬇️ Cloning BirdGuard repository...")
result = _subprocess.run(["git", "clone", repo_url, repo_dir])
if result.returncode != 0:
    print("❌ Failed to clone repository. Please check internet access or the repo URL.")
else:
    print("✅ Repository cloned into:", repo_dir)

# Model path (as used in the Pi scripts)
MODEL_PATH = Path(repo_dir) / "pimodelv1.1.pt"

if not MODEL_PATH.is_file():
    print("❌ Model file 'pimodelv1.1.pt' not found at:", MODEL_PATH)
else:
    print("✅ Found model file at:", MODEL_PATH)

⬇️ Cloning BirdGuard repository...
✅ Repository cloned into: BirdGuard
✅ Found model file at: BirdGuard/pimodelv1.1.pt


## 3. Model Classes & LED Logic

The deployment scripts use **Ultralytics YOLO** with:

- `cls == 1` → **Illegal bird**  
- `cls == 0` → **Legal bird**  
- Other class IDs are **ignored** for LED decisions.  

They define:

```python
CONF_TRIGGER = 0.10
REQUIRED_ON_TIME = 1   # seconds of continuous detection
```

In the live Pi system, a YOLO thread sets `detected_now = 1` when a detection satisfies the class + confidence condition.

The main loop then turns the GPIO LED ON only if `detected_now` stays true for **1 full second**.

### Static Image Adaptation (Notebook)

For a single uploaded image, there is no time axis, so we simplify to:

1. Run YOLO once with `conf = CONF_TRIGGER`.  
2. Inspect all detections in `results[0].boxes`.  
3. Apply BirdGuard LED rules:

   - If any detection has `cls == 1` and `conf ≥ CONF_TRIGGER`  
     → LED = 🔴 *blinking red* (**Illegal detected**).  
   - Else, if any detection has `cls == 0` and `conf ≥ CONF_TRIGGER`  
     → LED = 🟢 *solid green* (**Legal detected**, no illegal).  
   - Else  
     → LED = ⚪ *gray* (**no confident legal/illegal detection**).  

## 4. Load the YOLO Model

This cell loads the **exact same YOLO model** used on the Raspberry Pi:

```python
model = YOLO("pimodelv1.1.pt")
```

from the cloned repository.

In [ ]:
# 4. Load the YOLO model
# ----------------------

CONF_TRIGGER_DEFAULT = 0.10  # same as Pi scripts

if "MODEL_PATH" in globals() and MODEL_PATH.is_file():
    try:
        model = YOLO(str(MODEL_PATH))
        print("✅ YOLO model loaded from:", MODEL_PATH)
        try:
            model.info(verbose=True)
        except Exception:
            print("ℹ️ Model summary not available with this Ultralytics version.")
    except Exception as e:
        print("❌ Failed to load YOLO model.")
        print("Error:", e)
else:
    print("❌ MODEL_PATH is invalid, cannot load model.")

✅ YOLO model loaded from: BirdGuard/pimodelv1.1.pt
Model summary: 129 layers, 3,011,238 parameters, 0 gradients, 8.2 GFLOPs


## 5. LED Indicator (CSS Animation)

To emulate the Raspberry Pi LED:

- **Illegal detected** → animated, blinking **red** LED  
- **Legal detected (no illegal)** → solid **green** LED  
- **No confident detection** → neutral **gray** LED  

In [ ]:
# 5. LED CSS helper
# -----------------

def make_led_html(mode: str, text: str) -> str:
    """
    mode: 'illegal', 'legal', or 'none'
    text: status message
    """
    css = """
    <style>
    .led-container {
        display: flex;
        align-items: center;
        gap: 10px;
        font-family: Arial, sans-serif;
        font-size: 15px;
    }
    .led-base {
        width: 26px;
        height: 26px;
        border-radius: 50%;
        border: 2px solid #222;
    }
    .led-red {
        background: #ff0033;
        box-shadow: 0 0 12px #ff0033;
        animation: blink-red 0.8s infinite;
    }
    .led-green {
        background: #00cc44;
        box-shadow: 0 0 12px #00cc44;
    }
    .led-gray {
        background: #777777;
        box-shadow: 0 0 8px #777777;
    }
    @keyframes blink-red {
        0%   { opacity: 1; }
        50%  { opacity: 0.2; }
        100% { opacity: 1; }
    }
    </style>
    """

    if mode == "illegal":
        led_class = "led-base led-red"
    elif mode == "legal":
        led_class = "led-base led-green"
    else:
        led_class = "led-base led-gray"

    html = f"""
    {css}
    <div class=\"led-container\">
      <div class=\"{led_class}\"></div>
      <div><b>{text}</b></div>
    </div>
    """
    return html

## 6. Upload an Image & Run BirdGuard

This section creates the user interface:

- **File upload widget** (single image)  
- **Confidence threshold slider** (`CONF_TRIGGER`, default 0.10)  
- **“Run BirdGuard”** button  
- **LED status panel**  
- **Text summary** of detections (class IDs and confidences)  
- **Displayed image** for context  

LED decision logic:

- If any `cls == 1` with `conf ≥ CONF_TRIGGER` → 🔴 blinking **red**  
- Else if any `cls == 0` with `conf ≥ CONF_TRIGGER` → 🟢 solid **green**  
- Else → ⚪ **gray**  

In [ ]:
# 6. UI & inference
# -----------------

import io

upload_widget = widgets.FileUpload(
    accept="image/*",
    multiple=False,
    description="Upload image"
)

run_button = widgets.Button(
    description="Run BirdGuard",
    button_style="success",
    icon="play"
)

conf_slider = widgets.FloatSlider(
    value=CONF_TRIGGER_DEFAULT,
    min=0.05,
    max=0.9,
    step=0.05,
    description="CONF_TRIGGER",
    continuous_update=False,
)

led_widget = widgets.HTML(
    value=make_led_html("none", "Waiting for image...")
)

output_area = widgets.Output()

ui = widgets.VBox([
    widgets.HTML("<h3>📤 Step 1 — Upload a bird image</h3>"),
    upload_widget,
    widgets.HTML("<h3>🎚️ Step 2 — (Optional) adjust CONF_TRIGGER</h3>"),
    conf_slider,
    widgets.HTML("<h3>▶️ Step 3 — Run BirdGuard</h3>"),
    run_button,
    widgets.HTML("<hr>"),
    widgets.HTML("<h3>💡 LED status</h3>"),
    led_widget,
    widgets.HTML("<hr>"),
    widgets.HTML("<h3>📝 Detection summary & image</h3>"),
    output_area
])

display(ui)


def on_run_button_clicked(_b):
    output_area.clear_output()

    if not upload_widget.value:
        led_widget.value = make_led_html("none", "Please upload an image first.")
        return

    # Read uploaded image
    upload_data = next(iter(upload_widget.value.values()))
    img_bytes = upload_data["content"]
    filename = upload_data.get("metadata", {}).get("name", "uploaded_image")

    pil_img = Image.open(io.BytesIO(img_bytes)).convert("RGB")
    np_img = np.array(pil_img)

    conf_trigger = float(conf_slider.value)

    # Run YOLO (static-image version of the Pi loop)
    results = model.predict(
        np_img,
        imgsz=480,
        conf=conf_trigger,
        iou=0.45,
        device="cpu",
        verbose=False
    )

    boxes = results[0].boxes

    illegal_found = False
    legal_found = False
    detections = []

    if boxes is not None and len(boxes) > 0:
        for det in boxes:
            cls_id = int(det.cls[0])
            conf = float(det.conf[0])
            detections.append((cls_id, conf))

            if cls_id == 1 and conf >= conf_trigger:
                illegal_found = True
            elif cls_id == 0 and conf >= conf_trigger:
                legal_found = True

    # LED decision (mirrors Pi class+conf logic; no timing in static demo)
    if illegal_found:
        mode = "illegal"
        text = "⚠️ Illegal bird detected (class 1, conf ≥ CONF_TRIGGER)"
    elif legal_found:
        mode = "legal"
        text = "✔️ Legal bird detected (class 0, conf ≥ CONF_TRIGGER)"
    else:
        mode = "none"
        text = "No confident legal/illegal detection."

    led_widget.value = make_led_html(mode, text)

    # Show summary + image
    with output_area:
        print(f"Image: {filename}")
        print(f"CONF_TRIGGER used: {conf_trigger:.2f}")
        print("\nDetections (class_id, confidence):")
        if detections:
            for cls_id, conf in detections:
                print(f"  - class {cls_id} | conf = {conf:.3f}")
        else:
            print("  (no bounding boxes above this detection threshold)")

        plt.figure(figsize=(5,5))
        plt.imshow(pil_img)
        plt.axis("off")
        plt.title("Uploaded image (for visual reference)")
        plt.show()


run_button.on_click(on_run_button_clicked)

print("✅ Interface ready. Upload an image and click 'Run BirdGuard'.")

✅ Interface ready. Upload an image and click 'Run BirdGuard'.
